In [1]:
using Plots
using LinearAlgebra
using Revise
using Optim
using Zygote
using LaTeXStrings
using MultiFloats
using SparseIR
using Nevanlinna

┌ Info: Precompiling Nevanlinna [0b21cfa8-e591-48a4-a07b-a9d20be7c264]
└ @ Base loading.jl:1662


In [2]:
T = BigFloat

BigFloat

In [3]:
beta::Float64 = 100
wmax::Float64 = 100
basis = FiniteTempBasisSet(Float64(beta), Float64(wmax), 1e-15) 

FiniteTempBasisSet with β = 100.0, ωmax = 100.0

In [4]:
include("../flex_src/IRFLEX.jl")

Main.IRFLEX

In [6]:
hnw = (length(basis.smpl_wn_b.sampling_points)-1)÷2

52

In [13]:
input_chi  = Array{Complex{T}}(undef, hnw) 
input_smpl = Array{Complex{T}}(undef, hnw) 
;

In [38]:
f = open("FLEX_chi0.dat", "r")
sum_rule = parse(Float64,readline(f))
for i in 1:hnw
    list = readline(f)
    s  = split(list,'\t')
    o  = parse(BigFloat, s[1])
    re = parse(BigFloat, s[2])
    ii = parse(BigFloat, s[3])
    input_smpl[i] = im*o
    input_chi[i]  = -re - ii*im
end
close(f)

In [37]:
setprecision(512)

512

In [39]:
N_real    =  6000
eta       =  0.001
omega_max =  1.0+eta
H_max     =  50
ab_coeff  = zeros(ComplexF64, 2*H_max) 
lambda    = 1e-5
#1000 → 2000
iter_tol  = 10000
N_imag    =  Nevanlinna.calc_opt_N_imag(hnw, input_smpl, input_chi)
#N_imag = hnw 
println("N_imag $(N_imag)")

N_imag is setted as 22
N_imag 22


In [40]:
@time sol = Nevanlinna.NevanlinnaSolver(N_imag, input_smpl, input_chi, N_real, omega_max, eta, sum_rule, H_max, iter_tol, lambda, true)
;

Pick matrix is non positive semi-definite matrix in Schur method.
H=1
Iter     Function value   Gradient norm 
     0     7.692319e+00     3.610791e+01
 * time: 0.01637411117553711
     1     5.959521e+00     1.376888e+01
 * time: 5.832335948944092
     2     3.237334e+00     8.330503e+00
 * time: 9.7257719039917
     3     2.580651e+00     2.308128e+00
 * time: 11.68789792060852
     4     2.553472e+00     2.065056e-01
 * time: 13.637442111968994
     5     2.553353e+00     1.044147e-01
 * time: 14.896047115325928
     6     2.553319e+00     8.362800e-03
 * time: 16.168946981430054
     7     2.553318e+00     4.325013e-03
 * time: 18.107789993286133
     8     2.553318e+00     3.286115e-03
 * time: 19.3800630569458
     9     2.553318e+00     6.683691e-04
 * time: 20.666630029678345
    10     2.553318e+00     2.367071e-04
 * time: 22.006797075271606
    11     2.553318e+00     1.670284e-04
 * time: 23.893304109573364
    12     2.553318e+00     6.386005e-05
 * time: 25.23860192298889

In [41]:
@time Nevanlinna.solve!(sol)
;

H=1
Iter     Function value   Gradient norm 
     0     2.553318e+00     1.000847e-08
 * time: 0.0002009868621826172
     1     2.553318e+00     1.348790e-08
 * time: 1.248737096786499
     2     2.553318e+00     7.680359e-09
 * time: 2.5294790267944336
max_theta=0.255842386920734370004563853911877928782452084639885716857863251121064675121182769863549230087663306132249825535280713934046959677407336899136834495365516875
hardy optimization was success.
H=2
Iter     Function value   Gradient norm 
     0     2.553318e+00     1.365707e+01
 * time: 0.00019598007202148438
     1     1.994574e+00     1.128237e+01
 * time: 5.025259971618652
     2     6.785222e-01     3.574534e+00
 * time: 7.84345006942749
     3     6.427173e-01     2.790530e+00
 * time: 9.965175867080688
     4     9.266554e-02     2.505030e+00
 * time: 12.101970911026001
     5     6.704932e-02     3.796254e-02
 * time: 14.268976926803589
     6     6.679684e-02     2.688488e-02
 * time: 16.341472864151
     7     6.678261e

LoadError: InterruptException:

In [40]:
imaginary = Nevanlinna.ImagDomainData(input_smpl, input_chi, N_imag)
raw_reals = Nevanlinna.RealDomainData(N_real, omega_max, eta, sum_rule, T=T)
println(typeof(imaginary))
println(typeof(raw_reals))

phis = Nevanlinna.calc_phis(imaginary)
abcd = Nevanlinna.calc_abcd(imaginary, raw_reals, phis)
hardy_matrix = Nevanlinna.calc_hardy_matrix(raw_reals, H_max)

Nevanlinna.evaluation!(raw_reals, abcd, H_max, ab_coeff, hardy_matrix)

Pick matrix is positive semi-definite.
ImagDomainData{BigFloat}
RealDomainData{BigFloat}
max_theta=0.0
hardy optimization was success.


true

In [59]:
f = open("raw_spec.dat", "w")
for iw in 1:length(raw_reals.freq)
    omega = real(raw_reals.freq[iw])
    chi = imag(raw_reals.val[iw]/pi)*tanh(0.5*SparseIR.β(basis)*real(raw_reals.freq[iw]))
    println(f, "$omega \t $chi")
end
close(f)
;

In [ ]:
f = open("opt_spec.dat", "w")
for iw in 1:length(sol.reals.freq)
    omega = Float64(real(sol.reals.freq[iw]))
    chi = Float64(imag(sol.reals.val[iw]/pi))*tanh(0.5*SparseIR.β(basis)*real(sol.reals.freq[iw]))
    println(f, "$omega \t $chi")
end
close(f)
;